# Behavior cloning flow

Create the full flow for training a model for behavior cloning. This notebook programmatically generates a new set of exp/runs that cover all the necessary components for a behavior cloning system (sensor processing, behavior cloning training and verification notebooks)

It writes the exp/runs into an external directory fully separated from the github source, and creates an automation script that runs them. A separate directory for results is also created. 

Finally, it runs the necessary notebooks to execute the whole flow using papermill.

The results directory contain the output of this flow, both in terms of trained models, as well as results (in the verification exp/run).

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import pathlib
import yaml
import tqdm
import papermill
# from automate import automate_exprun
import bc_factory 
from demonstration.demonstration import list_demos
from demonstration.demopack import import_demopack, group_chooser_sp_bc_standard

# Setting up the flow directory and the demonstrations used as data
Setting up a separate directory for the flow, which includes an ```exprun``` directory for the generated exp/run config files and a ```results``` directory where the data goes. 

In [ ]:
# The primary parameters. These are the ones that one most likely want to change 

flow_name = "BC-touch-apple"
demopack_name = "touch-apple"
demonstration_cam = "dev0"
# demopack_name = "random-both-cameras-video"
# demopack_name = "automove-pack-01"
# demonstration_cam = "dev2"

# The number of epochs used for training.
# -- low values for debugging the flow
# epochs_sp = 15
# epochs_bc = 15
# -- realistic values determined experimentally
epochs_sp = 300
epochs_bc = 500

exprun_path, result_path = bc_factory.external_setup(flow_name, Config()["flows_path"])

demopack_path = pathlib.Path(Config()["demopacks_path"], demopack_name)
selection = import_demopack(demopack_path, group_chooser_sp_bc_standard)


***ExpRun**: Loading pointer config file:
	C:\Users\lotzi\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	c:\Users\lotzi\Work\_Config\BerryPicker\cfg\settings.yaml
***Path for external experiments:
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\exprun
***Path for external data:
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\result
***ExpRun**: Experiment config path changed to c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\exprun
***ExpRun**: Experiment data path changed to c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\result
***ExpRun**: Experiment demonstration copied to
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\exprun\demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\exprun\sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch

In [3]:
# the latent size used throughout this experiment
latent_size = 128

# FIXME: set the training data through some data structure passed as parameter....
# setting the training data
# cam = "dev0"
#run_demo = "touch-apple"
experiment = "demonstration"
# run = "freeform"
# run = "random-both-cameras"
# run = "random-both-cameras-video"
exp = Config().get_experiment(experiment, demopack_name)
demos = list_demos(exp)
print("***The demos considered")
pprint.pprint(demos)

sp_training_data = [[demopack_name, demo, demonstration_cam] for demo in selection["sp_training"]]
sp_validation_data = [[demopack_name, demo, demonstration_cam] for demo in selection["sp_validation"]]
bc_training_data = [[demopack_name, demo, demonstration_cam] for demo in selection["bc_training"]]
bc_validation_data = [[demopack_name, demo, demonstration_cam] for demo in selection["bc_validation"]]


# sp_training_data = []
# temp = list_demos(exp, "sp_training")

# for demo in list_demos(exp, "sp_training"):
#     sp_training_data.append([demopack_name, demo, demonstration_cam])
# print("***sp_training_data")
# pprint.pprint(sp_training_data)

# sp_validation_data = []
# for demo in list_demos(exp, "sp_validation"):
#     sp_validation_data.append([demopack_name, demo, demonstration_cam])

# bc_training_data = []
# for demo in list_demos(exp, "bc_training"):
#     bc_training_data.append([demopack_name, demo, demonstration_cam])

# bc_validation_data = []
# for demo in list_demos(exp, "bc_validation"):
#     bc_validation_data.append([demopack_name, demo, demonstration_cam])


***ExpRun**: Configuration for exp/run: demonstration/touch-apple successfully loaded
***The demos considered
['bc_testing_00000',
 'bc_testing_00001',
 'bc_testing_00002',
 'bc_training_00000',
 'bc_training_00001',
 'bc_training_00002',
 'bc_training_00003',
 'bc_validation_00000',
 'bc_validation_00001',
 'sp_testing_00000',
 'sp_testing_00001',
 'sp_testing_00002',
 'sp_training_00000',
 'sp_training_00001',
 'sp_training_00002',
 'sp_training_00003',
 'sp_validation_00000',
 'sp_validation_00001']


In [4]:
def generate_sensorprocessing_conv_vae(exprun_path, result_path, params, verify=False):
    """Generate the experiment for the conv-vae sensorprocessing with the right training data and parameters. Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. 
    If verify is true, it generates for the same run, with the verify notebook."""
    val = {}
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_exprun_path(), params["exp_sp"], params["run_sp"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    if not verify:
        v["name"] = f"TrainVAE-{path}"
        v["notebook"] = "sensorprocessing/Train_Conv_VAE.ipynb"
    else:
        v["name"] = f"VerifyVAE-{path}"
        v["notebook"] = "sensorprocessing/Verify_Conv_VAE.ipynb"
    vparams = {}
    vparams["experiment"] = params["exp_sp"]
    vparams["run"] = params["run_sp"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_sp"], "runname": params["run_sp"], "automation_entry": v}

In [5]:
def generate_bc_train(exprun_path, result_path, params):
    """Generate the experiment for the training of behavior cloning with the right training data and parameters. At the same time, it also generates the corresponding bcrun experiment, which will be used to drive the robot.  
    """    
    val = {}
    val["name"] = params["name"]
    val["exp_sp"] = params["exp_sp"]
    val["run_sp"] = params["run_sp"]
    # these had not yet been studied for alternatives
    val["sequence_length"] = params["sequence_length"]
    val["loss"] = params["loss"]
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]

    if params["algorithm"] == "lstm-mdn": 
        val["exp_mdn"] = "behavior_cloning"
        val["run_mdn"] = "mdn_for_bc_00"
        val["controller"] = "bc_LSTM_MDN"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm":
        val["controller"] = "bc_LSTM"
        val["controller_file"] = "controller.pth"
        val["num_layers"] = params["num_layers"]
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "lstm-residual":
        val["controller"] = "bc_LSTM_Residual"
        val["controller_file"] = "controller.pth"
        val["hidden_size"] = params["hidden_size"]
    elif params["algorithm"] == "mlp":
        val["controller"] = "bc_MLP"
        val["controller_file"] = "controller.pth"
        val["hidden_layers"] = params["hidden_layers"]
        val["hidden_layer_1"] = params["hidden_layer_1"]
        val["hidden_layer_2"] = params["hidden_layer_2"]
        
    else:
        raise Exception(f"*** generate_behaviorcloning: algorithm {params['algorithm']} not yet supported")
    
    val["control_size"] = 6 
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]

    # save the generated exprun spec
    path = pathlib.Path(Config().get_exprun_path(), params["exp_bc"], params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Train_BehaviorCloning.ipynb"
    vparams = {}
    vparams["run"] = params["run_bc"]
    vparams["experiment"] = params["exp_bc"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    # now, generate the bcrun experiment file, which is the same with the "runbc_" prefix
    # FIXME: these need to be configurable
    valrun = {}
    valrun["name"] = f"generated_run_{params['run_bc']}"
    valrun["exp_robot_controller"] = "robot_al5d"
    valrun["run_robot_controller"] = "position_controller_00"
    valrun["exp_camera_controller"] = "controllers"
    valrun["run_camera_controller"] = "camera_cam0_controller"
    valrun["control_camera"] = "dev0"
    valrun["exp_bc"] = "behavior_cloning"
    valrun["run_bc"] = "bc_lstm_0001"

    path = pathlib.Path(Config().get_exprun_path(), params["exp_bc"], "runbc_"+ params["run_bc"] + ".yaml")
    with open(path, "w") as f:
        yaml.dump(valrun, f)

    return {"experiment": params["exp_bc"], "runname": params["run_bc"], "automation_entry": v}

In [6]:
def generate_bc_verify(exprun_path, result_path, params):
    """Generate the experiment for the behavior cloning with the right training data and parameters. At the same time, it also generates the corresponding bcrun experiment, which will be used to drive the robot.  
    """    
    val = {}
    val["exp_sp"] = params["exp_sp"]
    val["run_sp"] = params["run_sp"]
    val["exp_bc"] = params["exp_bc"]
    val["run_bc"] = params["run_bc"]
    val["exp_bc_verify"] = params["exp_bc_verify"]
    val["run_bc_verify"] = params["run_bc_verify"]
    val["verification_data"] = params["verification_data"]

    # save the generated exprun spec
    path = pathlib.Path(Config().get_exprun_path(), params["exp_bc"], params["run_bc"] + "_verify.yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Verify_BehaviorCloning.ipynb"
    vparams = {}
    vparams["run"] = params["run_bc_verify"]
    vparams["experiment"] = params["exp_bc_verify"]
    vparams["external_path"] = exprun_path.as_posix()
    vparams["data_path"] = result_path.as_posix()
    v["params"] = vparams

    return {"experiment": params["exp_bc_verify"], "runname": params["run_bc_verify"], "automation_entry": v}

### Generate a range of exp/runs to be run

define which demonstrations will be used for:
* training the sensor processing (sp_training_data)
* training the behavior cloning (bc_training_data)
* verifying the behavior cloning (bc_verification_data) 
   etc.

In [7]:


# generate sensor processing expruns
expruns = []

exp_sp = "sensorprocessing_conv_vae"
run_sp = "_flow_sp_conv_vae_0001"
params = {}
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["epochs"] = epochs_sp
params["training_data"] = sp_training_data
params["validation_data"] = sp_validation_data
exprun = generate_sensorprocessing_conv_vae(
    exprun_path = exprun_path, result_path = result_path, params = params)
expruns.append(exprun)
# also the verify model for the same
exprun = generate_sensorprocessing_conv_vae(
    exprun_path = exprun_path, result_path = result_path, params = params, verify=True)
expruns.append(exprun)


# generate the behavior cloning expruns to match with the sensor processing components from above

exp_bc = "behavior_cloning"

# an MLP model
params = {}
params["name"] = "MLP 2 hidden" # should be descriptive
params["exp_bc"] = exp_bc
params["run_bc"] = "_flow_bc_mlp_0001"
params["algorithm"] = "mlp"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_layers"] = 2
params["hidden_layer_1"] = 50
params["hidden_layer_2"] = 20
params["epochs"] = epochs_bc
params["sequence_length"] = 1
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the MLP model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "_flow_bc_mlp_0001_verify"
paramsv["verification_data"] = bc_validation_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)


# a pure LSTM model
params = {}
params["name"] = "LSTM 2 layers"
params["exp_bc"] = exp_bc
params["run_bc"] = "_flow_bc_lstm_0001"
params["algorithm"] = "lstm"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["num_layers"] = 2
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "_flow_bc_lstm_0001_verify"
paramsv["verification_data"] = bc_validation_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)

# a residual LSTM model
params = {}
params["name"] = "LSTM residual"
params["exp_bc"] = exp_bc
params["run_bc"] = "_flow_bc_lstm_residual_0001"
params["algorithm"] = "lstm-residual"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "_flow_bc_lstm_residual_0001_verify"
paramsv["verification_data"] = bc_validation_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)

# an LSTM-MDN based model
params = {}
params["name"] = "LSTM MDN"
params["exp_bc"] = exp_bc
params["run_bc"] = "_flow_bc_lstm_mdn_0001"
params["algorithm"] = "lstm-mdn"
params["exp_sp"] = exp_sp
params["run_sp"] = run_sp
params["latent_size"] = latent_size
params["hidden_size"] = 32
params["epochs"] = epochs_bc
params["sequence_length"] = 10
params["loss"] = "MSELoss"
params["training_data"] = bc_training_data
params["validation_data"] = bc_validation_data
exprun = generate_bc_train(exprun_path=exprun_path, result_path=result_path, params=params)
expruns.append(exprun)
# the verification for the LSTM-MDN model
paramsv = {}
paramsv["exp_bc"] = params["exp_bc"]
paramsv["run_bc"] = params["run_bc"]
paramsv["exp_sp"] = exp_sp
paramsv["run_sp"] = run_sp
paramsv["exp_bc_verify"] = exp_bc
paramsv["run_bc_verify"] = "_flow_bc_lstm_mdn_0001_verify"
paramsv["verification_data"] = bc_validation_data
exprun = generate_bc_verify(exprun_path=exprun_path, result_path=result_path, params=paramsv)
expruns.append(exprun)


### Generate an automation

Generate an exp/run of the type "automate" based on the list expruns. The individual automation entries should be already in the "automation_entry" field. The only thing that is really decided here is the "creation_style", which can be exist-ok or discard-old, if we want to start this from scratch. 

In [8]:
value = {}
val = []
value["exps_to_run"] = val

# Use exist_ok not to-re-run previously successfully run models
creation_style = "exist-ok"
# creation_style = "discard-old"

for exprun in expruns:
    v = exprun["automation_entry"]
    if "verify" in v["params"]["run"]:
        v["params"]["creation_style"] = "discard-old"
    else:
        v["params"]["creation_style"] = creation_style
    val.append(v)

path = pathlib.Path(Config().get_exprun_path(), "automate", "_flow_bc.yaml")
with open(path, "w") as f:
    yaml.dump(value, f)

### Run the automation script

Run the automation script, as a series of notebooks. In order to follow the execution inside these notebooks, one needs to open the output notebook, which is in the output_filename. 

* FIXME: this should be done such that I can also run it from command line

In [9]:
experiment = "automate"
run = "_flow_bc"
exp = Config().get_experiment(experiment, run)

print(f"***Starting automated running of the flow.\n The path for the output notebooks is\n{result_path}")

for item in tqdm.tqdm(exp["exps_to_run"]):
    print(f"***Automating {item['notebook']} :\n {item['params']['experiment']}/{item['params']['run']}")
    notebook_path = pathlib.Path("..", item["notebook"])
    output_filename = f"{notebook_path.stem}_{item['params']['experiment']}_{item['params']['run']}_output{notebook_path.suffix}"
    print(f"--> {output_filename}")
    # pprint.pprint(item["params"])
    output_path = pathlib.Path(result_path, output_filename)
    try:
        papermill.execute_notebook(
            notebook_path,
            output_path.absolute(),
            cwd=notebook_path.parent,
            parameters=item["params"]
        )
    except Exception as e:
        print(f"There was an exception {e}")    

***ExpRun**: Experiment default config c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\exprun\automate\_defaults_automate.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: automate/_flow_bc successfully loaded
***Starting automated running of the flow.
 The path for the output notebooks is
c:\Users\lotzi\Work\_Data\BerryPicker-Flows\BC-touch-apple\result


  0%|          | 0/10 [00:00<?, ?it/s]

***Automating sensorprocessing/Train_Conv_VAE.ipynb :
 sensorprocessing_conv_vae/_flow_sp_conv_vae_0001
--> Train_Conv_VAE_sensorprocessing_conv_vae__flow_sp_conv_vae_0001_output.ipynb


Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

 10%|█         | 1/10 [01:09<10:28, 69.82s/it]

...

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

 90%|█████████ | 9/10 [02:42<00:14, 14.63s/it]

***Automating behavior_cloning/Verify_BehaviorCloning.ipynb :
 behavior_cloning/_flow_bc_lstm_mdn_0001_verify
--> Verify_BehaviorCloning_behavior_cloning__flow_bc_lstm_mdn_0001_verify_output.ipynb


Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

100%|██████████| 10/10 [03:03<00:00, 18.36s/it]
